# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import hvplot.pandas
import geoviews as gv
from geoviews import opts
from geoviews.tile_sources import EsriImagery

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,17.51,75,23,7.36,AU,1733026505
1,1,iqaluit,63.7506,-68.5145,-8.15,61,75,4.12,CA,1733026506
2,2,grytviken,-54.2811,-36.5092,1.29,75,56,3.54,GS,1733026507
3,3,jamestown,42.0970,-79.2353,-5.55,68,20,5.66,US,1733026508
4,4,stanley,54.8680,-1.6985,11.01,90,20,6.69,GB,1733026509


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# Create a map that shows each city as a point, with point size representing humidity
city_map = city_data_df.hvplot.points(
    "Lng",  # Longitude for x-axis
    "Lat",  # Latitude for y-axis
    geo=True,  # Enable geographical plotting
    size=10,  # Size of the points
    color="City",  # Unique color for each city
    tiles="EsriImagery",  # Use a geographic tile background
    alpha=0.7,  # Set transparency
    frame_width=800,  # Adjust map width
    frame_height=600,  # Adjust map height
    title="Cities Map Visualization"  # Title for the map
)

# Show the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &  # Max temperature greater than 21 degrees
    (city_data_df["Max Temp"] < 27) &  # Max temperature less than 27 degrees
    (city_data_df["Wind Speed"] < 4.5) &  # Wind speed less than 4.5 m/s
    (city_data_df["Cloudiness"] == 0)  # Cloudiness equals 0
]

# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,tabou,4.4230,-7.3528,25.35,84,0,2.12,CI,1733026558
125,125,wailua homesteads,22.0669,-159.3780,23.44,85,0,3.60,US,1733026649
141,141,lihue,21.9789,-159.3672,23.77,81,0,3.60,US,1733026666
165,165,beitbridge,-22.2167,30.0000,21.73,48,0,2.87,ZW,1733026694
254,254,kapa'a,22.0752,-159.3190,24.18,84,0,3.60,US,1733026792


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create a new DataFrame with required columns
hotel_df = criteria_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column for "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
47,tabou,CI,4.4230,-7.3528,84,
125,wailua homesteads,US,22.0669,-159.3780,85,
141,lihue,US,21.9789,-159.3672,81,
165,beitbridge,ZW,-22.2167,30.0000,48,
254,kapa'a,US,22.0752,-159.3190,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000  # Search within 10,000 meters
geoapify_key = geoapify_key
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,  # Return only the first result
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
tabou - nearest hotel: hôtel le rochet
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
lihue - nearest hotel: Kauai Palms
beitbridge - nearest hotel: Beitbridge Inn & Casino
kapa'a - nearest hotel: Pono Kai Resort
praia - nearest hotel: Casa Sodadi
caucete - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
cidade velha - nearest hotel: Encosta Azul


,City,Country,Lat,Lng,Humidity,Hotel Name
47,tabou,CI,4.4230,-7.3528,84,hôtel le rochet
125,wailua homesteads,US,22.0669,-159.3780,85,Hilton Garden Inn Kauai Wailua Bay
141,lihue,US,21.9789,-159.3672,81,Kauai Palms
165,beitbridge,ZW,-22.2167,30.0000,48,Beitbridge Inn & Casino
254,kapa'a,US,22.0752,-159.3190,84,Pono Kai Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",  # Longitude for x-axis
    "Lat",  # Latitude for y-axis
    geo=True,  # Enable geographic plotting
    size=10,  # Adjust the size of the points
    color="City",  # Unique color for each city
    tiles="EsriImagery",  # Background tiles with a map
    frame_width=800,  # Set map width
    frame_height=600,  # Set map height
    hover_cols=["City", "Country", "Hotel Name"]  # Add hover information
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)